In [1]:
import pandas as pd
from konlpy.tag import Okt
from collections import Counter


In [2]:
def filesave(dataframe,name):
    dataframe.to_csv(name,sep="\t", encoding="utf-8",index=False)
def fileread(name):
    return pd.read_csv(name,sep="\t",encoding="utf-8")

In [3]:
test=fileread('joongang_dataframe.csv')

In [4]:
test

,title,content,date,section,url
0,"“시장 왜곡돼 집값 상승세 지속, 내년 대선이 분수령”",\n 전문가 5인의 부동산 시장 전망 \n 11억930만원. 한국부동산원이 ...,2021.08.21 05:46,중앙SUNDAY,NaN
1,'저녁 있는 삶' 이런 카피 없나···섹시 고집하다 당한 與주자들,\n더불어민주당 대선 주자 이낙연 전 대표가 발표한 중산층경제 성장 전략 '삼중폭격...,2021.08.21 05:54,정치,NaN
2,"[단독]""대역 아니냐"" 확 달라진 전두환, 알고보니 혈액암",\n\n 전두환(90) 전 대통령이 혈액암의 일종인 다발성 골수종 진단을 받은 것...,2021.08.21 05:55,경제,NaN
3,美 Z세대가 中 쇼핑몰에 갇혔다…자라·H&M 제낀 쉬인SHEIN,\n팬데믹으로 패션업계가 위축된 가운데 중국 SPA 온라인 쇼핑몰 쉬인(Shein)...,2021.08.21 05:56,경제,NaN
4,"김의겸 선배 행복하십니까…외신이 묻네요 ""이게 무슨 법이냐"" [뉴스원샷]",\n18일 서울 여의도 국회에서 열린 '언론중재 및 피해구제 등에 관한 법률 일부 ...,2021.08.21 05:56,국제,NaN
...,...,...,...,...,...
110477,"""내가 정인이 아빠"" 절규한 유튜버…""후원금으로 게장 사먹어""",\n양부모 학대로 숨진 정인양을 위한다는 명목으로 후원금을 받은 유튜버가 횡령 혐의...,2022.10.19 22:07,사회,https://www.joongang.co.kr/article/25110479
110478,"당사 압수수색 반발하는 野에…與 ""당당하게 집행 받아들여야""",\n더불어민주당 의원들이 19일 서울 여의도 중앙당사 앞에서 민주연구원 압수수색에 ...,2022.10.19 22:12,정치,https://www.joongang.co.kr/article/25110480
110479,제42회 전국장애인체전 울산에서 개막,\n10월 19일 울산종합운동장에서 열린 제42회 전국장애인체육대회 개회식에서 마지...,2022.10.19 22:16,스포츠,https://www.joongang.co.kr/article/25110481
110480,"야생마 푸이그의 가을 질주…키움, PO행 100% 확률 잡았다",\n키움 야시엘 푸이그가 19일 KT 위즈와의 준PO 3차전에서 1회 선제 3점 홈...,2022.10.19 22:22,스포츠,https://www.joongang.co.kr/article/25110482


In [5]:
test.drop_duplicates(inplace=True, ignore_index=True)

In [6]:
len(test)

110482

In [7]:
test['content'].nunique()

109481

In [8]:
def save_sentence_dataframe(data):
    idx=0
    sentences = []
    section_temp = []
    for i in range(len(data)):
        content = data['content'].iloc[idx]
        if "▲" in str(content):
            temp = str(content).split("▲")
            content=temp[0]
        splited = str(content).replace(". ","\n").split("\n")
        sentences += splited
        section_temp += [data['section'].iloc[idx]]*len(splited)
        idx+=1
    df = pd.DataFrame([sentences,section_temp]).T
    df.columns = ["sentence","section"]
    filesave(df, "joogang_sentence_section.csv")

In [9]:
save_sentence_dataframe(test)

In [10]:
ttt=fileread("joogang_sentence_section.csv")

In [12]:
def topic_counter_by_section_from_sentence_csv(section,data):
    okt=Okt()
    parsed_data=[]
    sectiondata = data[data["section"]==section]
    for sentence in sectiondata['sentence']:
        if len(str(sentence))<6:
            continue
        nouns = okt.nouns(str(sentence))
        stopword = ["없다","있다","재밌다","좋다","최고","같다","이","것","진짜","아니다","이렇다","그냥","하나","다시","많다","이다","최악",'완전',"안되다",'어떻다',"정말","보고",'정도','역시']
        for word in nouns:
            if len(word)>1 and word not in stopword:
                parsed_data.append(word)
    counts = Counter(parsed_data)
    return counts

In [13]:
test["section"].value_counts()

사회                                                            27733
정치                                                            20133
경제                                                            17173
국제                                                            11462
브랜드뉴스                                                          8490
스포츠                                                            6151
오피니언                                                           5795
문화                                                             4319
중앙SUNDAY                                                       3226
피플                                                             1445
팩플                                                              642
여행레저                                                            607
더,오래                                                            568
라이프                                                             548
더 북한                                            

In [17]:
sections = test["section"].value_counts()

In [18]:
sections = list(sections[sections>60].keys())

In [19]:
sections

['사회',
 '정치',
 '경제',
 '국제',
 '브랜드뉴스',
 '스포츠',
 '오피니언',
 '문화',
 '중앙SUNDAY',
 '피플',
 '팩플',
 '여행레저',
 '더,오래',
 '라이프',
 '더 북한',
 '앤츠랩',
 'COOKING',
 'Mr.밀리터리',
 'hello! Parents',
 '비크닉']

In [20]:
word_count_df = pd.DataFrame(columns=['word','count','section'])

In [21]:
word_count_df

,word,count,section


In [137]:
# counter=topic_counter_by_section("경제",test)

KeyboardInterrupt: 

In [ ]:
# temp = pd.DataFrame([dict(counter)]).T
# temp['word'] = temp.index
# temp['section'] = "경제"
# temp.columns=['count','word','section']
# word_count_df = pd.concat([word_count_df,temp],ignore_index=True)

In [ ]:
# word_count_df

In [22]:
def make_word_count_df():
    df = pd.DataFrame(columns=['word','count','section'])
    for section in sections:
        print(section)
        counter=topic_counter_by_section(section,test)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [23]:
def make_word_count_df2(data,section_list):
    df = pd.DataFrame(columns=['word','count','section'])
    for section in section_list:
        print(section)
        counter=topic_counter_by_section_from_sentence_csv(section,data)
        temp = pd.DataFrame([dict(counter)]).T
        temp['word'] = temp.index
        temp['section'] = section
        temp.columns=['count','word','section']
        df = pd.concat([df,temp],ignore_index=True)
    return df

In [30]:
t=test[(test['section']=="Sports")&(test['content'].str.contains("경기"))]

In [25]:
cnt=0
for i in t['content']:
    print(i)
    
    cnt+=1
    if cnt>6:
        break

NameError: name 't' is not defined

In [ ]:
w2 = make_word_count_df2(ttt,['사회','정치','경제','국제','브랜드뉴스','스포츠','오피니언','문화','중앙SUNDAY','피플','팩플','여행레저','더,오래','라이프','더 북한','앤츠랩','COOKING','Mr.밀리터리','hello! Parents','비크닉'])

사회


In [ ]:
w2

In [ ]:
w2.sort_values(by="count", ascending=False)

In [ ]:
w2['word_length']=[len(i) for i in w2['word']]

In [ ]:
w2 = w2[w2['word_length']>1]

In [ ]:
w2.drop(['word_length'],axis=1,inplace=True)

In [ ]:
w2.sort_values(by="count", ascending=False)

In [ ]:
filesave(w2,"joogang_content_word_count.csv")